In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_excel(r"C:\Users\ajayp\Downloads\Final_clean_word_split_hindi.xlsx")

#df['splits']= np.zeros(len(df['Unnamed: 0']))
df  = df.fillna(' ')
#df.to_csv('C:\Users\ajayp\Downloads\Final_clean_word_split_hindi_1.xlsx')
#df.count(df['Unnamed: 2']==None)

In [ ]:
df['splits']=' '
for i in range(len(df['Unnamed: 0'])):
    df['splits'][i] = df['Unnamed: 2'][i]+'+'+df['Unnamed: 3'][i] +'+'+df['Unnamed: 4'][i]
    if df['Unnamed: 2'][i] == ' ':
        df['splits'][i] = df['Unnamed: 3'][i]+'+'+df['Unnamed: 4'][i]
    if df['Unnamed: 3'][i] == ' ':
        df['splits'][i] = df['Unnamed: 2'][i]+'+'+df['Unnamed: 4'][i] 
    if df['Unnamed: 4'][i] == ' ':
        df['splits'][i] = df['Unnamed: 2'][i]+'+'+df['Unnamed: 3'][i]
    #else:
   #     df['splits'][i] = df['Unnamed: 2'][i]+'+'+df['Unnamed: 3'][i] +'+'+df['Unnamed: 4'][i]  
    
df.head(n=100)

In [ ]:
def extract_unique_chars(words):
    unique_chars = set()
    for word in words:
        print(word)
        for char in word:

            unique_chars.add(char)
    return unique_chars
#df = df['stem1'].dropna()
all_words = df['Unnamed: 1'].tolist()#+df['stem1'].tolist()
print(len(all_words))
list1 = []
list2 = []
list3 = []
list4 = []
for i in df.index:
    list1.append(df['splits'][i])
#print(list1)
print(len(all_words))
unique_chars = extract_unique_chars(all_words)
unique_chars1= extract_unique_chars(list1)
char_vocab=[]
for i in unique_chars:
    char_vocab.append(i)
for i in unique_chars1:
    char_vocab.append(i)
char_vocab_1 = set(char_vocab)
print(f"Unique devanagari Characters: {unique_chars} {unique_chars1}")
print(len(char_vocab_1))
char_vocab_1

In [ ]:
import torch
def char_tensor(word, char_vocab):
    tensor = torch.zeros(len(word), len(char_vocab))
    default_char=' '
    for i, char in enumerate(word):
        if char in char_vocab:
            for j in range(len(char_vocab)):
                if char_vocab[j]==char:
                    tensor[i][j] = 1
        else:
            print(f"Warning: Character '{char}' not found in vocabulary. Using default index.")
            tensor[i][-1] = 1  # Use the default character
    return tensor
print(char_tensor('अङ्कफल',char_vocab))
#char_tensor('candra',char_vocab)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from tensorflow.keras.layers import Bidirectional

In [ ]:
input_texts = df['Unnamed: 1']
output_texts = df['splits']

input_texts_2 = df['Unnamed: 1']
output_texts_2 = df['splits']
k=0
#for i in X_train.index:
 #   input_texts[k] = char_tensor(X_train[i],char_vocab)
  ##  k+=1
# Tokenize the input and output texts
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(output_texts)
#input_sequences = np.zeros(max_seq_len)

input_sequences = tokenizer.texts_to_sequences(input_texts)
output_sequences = tokenizer.texts_to_sequences(output_texts)
max_seq_len = 15
input_sequences = pad_sequences(input_sequences, max_seq_len, padding ='post')
output_sequences = pad_sequences(output_sequences, max_seq_len, padding = 'post')

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X_one_hot = np.array([to_categorical(x, num_classes=len(char_vocab_1)) for x in input_sequences])
Y_one_hot = np.array([to_categorical(y, num_classes=len(char_vocab_1)) for y in output_sequences])

In [ ]:
input_size = 10  # Number of unique integers in the input
sequence_length = 5  # Length of the input sequences
hidden_units = 180  # Number of hidden units in the RNN
batch_size = 32  # Batch size for training
epochs = 20

In [ ]:
#model = Sequential([
#    SimpleRNN(100, input_shape=(15, len(char_vocab_1)), return_sequences=True),
#    TimeDistributed(Dense(input_size, activation='softmax'))
#])

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(X_one_hot, Y_one_hot, epochs=5, batch_size=320)

print(f"X_one_hot shape: {X_one_hot.shape}")  # Should be (num_samples, sequence_length, input_size)
print(f"Y_one_hot shape: {Y_one_hot.shape}")  # Should be (num_samples, sequence_length, input_size)

# Build the RNN model
model = Sequential([
    Bidirectional(LSTM(hidden_units, input_shape=(max_seq_len, 96), return_sequences=True)),
    Dropout(0.3),])
    #SimpleRNN(hidden_units, input_shape= hidden_units,return_sequences=True)),
    #Dropout(0.4),
    #TimeDistributed(Dense(96, activation='softmax'))])
#model.add(Bidirectinal(hidden_units))
model.add(Bidirectional(LSTM(hidden_units, return_sequences = True)))
model.add(Dropout(0.4))
model.add(TimeDistributed(Dense(96, activation='softmax')))



In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True)

# Train the model with validation split and early stopping
history = model.fit(
    X_one_hot, Y_one_hot,
    epochs=200,
    batch_size=batch_size,
    validation_split=0.2,
    #callbacks=[early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

plt.plot(history.history['val_accuracy'])
plt.title('Model validation accuracy')
plt.ylabel('val_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

plt.plot(history.history['val_loss'])
plt.title('Model val_loss')
plt.ylabel('val_loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

plt.plot

model.save('Bidirectional_Splitting_LSTM.h5')

In [ ]:
def generate_output_sequence(model, input_sequence):
    input_one_hot = to_categorical(input_sequence, num_classes=96)#.reshape((1, 15, 96))
   # input_one_hot = pad_sequences(input_one_hot,maxlen=15,padding='post')
    output_one_hot = model.predict(input_one_hot)
    output_sequence = np.argmax(output_one_hot, axis=2)
    
    return output_sequence.flatten()

list=[]
list2=[]
for input_sequence_1 in df['Unnamed: 1'][:10]:
    i = tokenizer.texts_to_sequences(input_sequence_1)
    i = pad_sequences(i,maxlen=15, padding='post')
    output_sequence_1 = generate_output_sequence(model, i)
    list2.append([input_sequence_1])
    list.append(tokenizer.sequences_to_texts([output_sequence_1]))
print(list)

In [ ]:
k=0
for i in tokenizer.sequences_to_texts(list):
    i=i.replace(' ', '')
    i=i.replace('++','+')
    i=i.replace('+++','+')
    i=i.replace('++++','+')
    i=i.replace('++','+')
    print(list2[k])
    print(i)
    k+=1
#print(a)

In [ ]:
df['predicted']=np.zeros((len(df['Unnamed: 1']),15))
k = model.predict(X_one_hot)



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_token_level_metrics(true_sequences, predicted_sequences):
    """
    Calculate token-level precision, recall, and F1 score for word splitting.

    Parameters:
    true_sequences (list of list of str): The ground truth token sequences.
    predicted_sequences (list of list of str): The predicted token sequences.

    Returns:
    dict: A dictionary containing precision, recall, and F1 score.
    """
    assert len(true_sequences) == len(predicted_sequences), "Number of sequences must match."

    # Flatten the lists
    true_tokens = [token for seq in true_sequences for token in seq]
    predicted_tokens = [token for seq in predicted_sequences for token in seq]

    # Create a binary representation of the presence of each token
    unique_tokens = list(set(true_tokens + predicted_tokens))
    true_binary = [[1 if token in seq else 0 for token in unique_tokens] for seq in true_sequences]
    predicted_binary = [[1 if token in seq else 0 for token in unique_tokens] for seq in predicted_sequences]

    # Flatten binary representations
    true_binary_flat = [item for sublist in true_binary for item in sublist]
    predicted_binary_flat = [item for sublist in predicted_binary for item in sublist]

    precision = precision_score(true_binary_flat, predicted_binary_flat, average='macro')
    recall = recall_score(true_binary_flat, predicted_binary_flat, average='macro')
    f1 = f1_score(true_binary_flat, predicted_binary_flat, average='macro')

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

# Example usage:
true_sequences = df['splits']
predicted_sequences = df['predicted'] 

metrics = calculate_token_level_metrics(true_sequences, predicted_sequences)
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1 Score: {metrics['f1_score']:.4f}")
